
* Nested try block
* User-defined Exceptions
* Logging
* Assertions
* Warnings
* Built-in Exceptions


---

## 1. Nested `try` Block



A **nested try block** is when you have a `try…except…finally` (or just `try…except`) inside another `try`. In Python, this allows you to handle errors at different “layers” of your code — inner-block handles specific issues, outer block handles more general or fallback issues. According to a tutorial: *“In a Python program, if there is another try-except construct either inside either a try block or inside its except block, it is known as a nested-try block.”* ([TutorialsPoint][1])

Why use it? Because sometimes you have an operation (outer) composed of sub-operations (inner) each of which may throw different exceptions and may require different handling. Nested blocks let you localize error handling. However: this can reduce readability and maintainability if overused. ([Software Engineering Stack Exchange][2])

### Code Examples

Let’s walk through some examples showing various scenarios.

#### Example A: Simple nested try

```python
a = 10
b = 0

try:
    print("Outer try: about to divide")
    try:
        result = a / b
        print("Inner try result:", result)
    except ZeroDivisionError as e:
        print("Inner except: division by zero:", e)
    finally:
        print("Inner finally: clean-up inner.")
    print("Back in outer try after inner try")
except Exception as outer_e:
    print("Outer except: caught something:", outer_e)
finally:
    print("Outer finally: always executes")
```

**What happens:**

* `a / b` triggers `ZeroDivisionError`.
* That is caught by the *inner* `except ZeroDivisionError`.
* `finally` of inner executes.
* Then control returns to outer try **after** the inner try block.
* Since the inner exception was handled, outer except does **not** execute (unless something else fails).
* Outer finally executes at the end.

#### Example B: Inner exception not handled → propagated to outer

```python
a = 10
b = 0

try:
    print("Outer try start")
    try:
        print("Inner try: about to divide")
        result = a / b
        print("Inner try result:", result)
    except KeyError:
        print("Inner except: key error caught")
    finally:
        print("Inner finally executed")
    print("Outer try after inner")
except ZeroDivisionError as e:
    print("Outer except: zero division caught:", e)
finally:
    print("Outer finally done")
```

Here, the inner except only catches `KeyError`, but division causes `ZeroDivisionError`. So:

* Inner except does **not** catch it → error propagates out of inner try.
* That means the outer try sees the exception and catches it in its except block.
* Then outer finally executes.
  This pattern is shown in the Tutorialspoint nested try block examples. ([TutorialsPoint][1])

### When to use & caveats

**Use it when:**

* You have operations arranged in layers and each layer has distinct failure modes.
* You want to handle some errors locally (inner) and others globally (outer).
* You need to ensure a cleanup at each layer.

**Be cautious because:**

* Too much nesting leads to confusing code flow.
* Catching overly generic exceptions in multiple layers can mask bugs. As one comment puts it: *“Nested try-except blocks can make the code harder to understand.”* ([Software Engineering Stack Exchange][2])
* It may be better to refactor into separate functions rather than deeply nested try blocks.
* Make sure your exceptions are specific and you aren’t catching `Exception` or bare `except:` indiscriminately.

### Best Practices

* Catch most specific exceptions you expect, not just generic `Exception`.
* Keep inner blocks small and clearly scoped.
* Avoid catching `BaseException` unless you have a very good reason.
* Use finally blocks to clean up resources (files, network sockets, etc).
* If nesting gets complicated, consider refactoring into functions.

---

## 2. User-Defined Exceptions



While Python comes with many built-in exception types, often your application has domain-specific error conditions. For this you can define your own exception classes. Typically you subclass `Exception` (or a more specific built-in exception) and optionally add custom attributes or logic. ([GeeksforGeeks][3])

Why? Because:

* It makes your code more expressive (you raise `InvalidAgeError` instead of generic `ValueError`).
* It allows catching your specific errors distinctly (e.g., `except InvalidAgeError:`).
* It helps with clarity and debugging.

### Code Example

```python
# Step 1: Define a custom exception class
class InvalidAgeError(Exception):
    def __init__(self, age, message="Age must be between 0 and 120"):
        self.age = age
        self.message = message
        super().__init__(self.message)

    def __str__(self):
        return f"[InvalidAgeError] {self.age} -> {self.message}"

# Step 2: Use it
def set_age(age):
    if not (0 <= age <= 120):
        raise InvalidAgeError(age)
    print(f"Age set to {age}")

# Step 3: Handle it
try:
    set_age(150)
except InvalidAgeError as e:
    print("Caught custom exception:", e)
```

Output:

```
Caught custom exception: [InvalidAgeError] 150 -> Age must be between 0 and 120
```

This example is taken directly from GeeksforGeeks. ([GeeksforGeeks][3])

You can also subclass built-in error types if you want a more specific category: e.g., `class NetworkError(RuntimeError): …`

### Key points and caveats

* Always inherit from `Exception` (not `BaseException`) unless you know what you are doing. ([Python documentation][4])
* Provide meaningful messages.
* Use custom attributes if you want to provide additional context (error codes, problematic value, metadata).
* Don’t over-complicate: if an existing built-in exception conveys your condition adequately, you might just use that instead of creating a new class.
* When catching, ensure you catch the most specific exceptions first (so your user-defined exception isn’t inadvertently caught by a more generic `except Exception:` above).

### Best Practices

* Define a common base exception for your application’s domain (e.g., `class AppError(Exception): pass`) and then subclass that for specific errors.
* When raising, always include enough context (value, operation, reason).
* Document your custom exceptions (what they mean, when they are raised).
* Avoid using custom exceptions where built-in ones suffice (e.g., using `ValueError` when it’s just a bad value).
* Use custom exceptions to distinguish “expected but error” states vs. “unexpected bug” states.

---

## 3. Logging



Logging is the practice of recording events that happen during a program’s execution — especially those of interest for debugging, auditing, or monitoring. Python’s standard library includes the `logging` module which provides a flexible framework for emitting log messages from Python programs. ([Python documentation][5])

Why log? Because:

* Instead of relying on `print()` statements (which are brittle, non-configurable, and often removed), logging gives you levels (DEBUG, INFO, WARNING, ERROR, CRITICAL), outputs to different places (console, file, remote), formatting, and control. ([Sematext][6])
* In exception handling context, logging allows you to record exception stack traces, context information, without necessarily terminating the program.
* For production systems, logging is essential for auditing and diagnosing problems after the fact.

### Code Example

```python
import logging

# Basic configuration: log to file, include timestamp
logging.basicConfig(
    filename='app.log',
    encoding='utf-8',
    level=logging.DEBUG,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

logger = logging.getLogger(__name__)

def divide(a, b):
    logger.debug(f"divide called with a={a}, b={b}")
    try:
        result = a / b
    except ZeroDivisionError as e:
        logger.error("Division by zero error", exc_info=True)
        raise  # re-raise if desired
    else:
        logger.info(f"Result is {result}")
        return result
    finally:
        logger.debug("divide ended")

if __name__ == "__main__":
    divide(10, 0)
```

**Explanation:**

* We configure logging to write to file `app.log`, set level to DEBUG so all levels are output. ([Python documentation][7])
* `logger.error(..., exc_info=True)` logs the stack trace of the exception.
* In production you might route different levels to different handlers (console vs file), use rotating logs, etc.

### Things to note / caveats

* Logging should be configured once (usually at program start). Subsequent modules/classes just get loggers (via `logging.getLogger(__name__)`) and emit messages. ([Stack Overflow][8])
* Logging levels: default root logger level is WARNING, so DEBUG and INFO might not appear unless you change the level. ([Real Python][9])
* Avoid heavy operations inside log message formatting when logging is disabled for that level (use lazy formatting or check level).
* Don’t use `print()` for production-level diagnostics. Use logging so you can control verbosity, output destination, format.
* For exception handling integration: in an `except` block you might log the error rather than just silent fail.
* Over-logging or logging sensitive information can create its own issues (performance, disk use, info exposure).

### Best Practices

* Use one logger per module: `logger = logging.getLogger(__name__)`.
* Configure logging at entry point, not in libraries. Libraries should assume logging is set up.
* Use appropriate levels: DEBUG for development detail, INFO for high-level events, WARNING for recoverable issues, ERROR for failures, CRITICAL for very severe.
* Include contextual information (e.g., who, what, where) in log messages.
* Use handlers and formatters for multi-destination logging (file, console, remote).
* Use `logger.exception()` inside exception handlers (equivalent to `error(..., exc_info=True)`).
* Avoid exposing sensitive data in logs (passwords, personal data).
* Consider log rotation or size limits to prevent log files growing unbounded.

---

## 4. Assertions

### Theory

An **assertion** is a statement that you believe to be true at a particular point in program execution; if the condition is false, you want the program to stop (typically during development) and raise an `AssertionError`. In Python the syntax is:

```python
assert condition, "optional error message"
```

([programiz.com][10])

Use of `assert` is intended for *developer checks* — ensuring internal logic invariants, preconditions/postconditions, etc. They are not a replacement for runtime error handling (exceptions) because in optimized Python mode (with the `-O` flag) assertion statements are removed. ([BrowserStack][11])

### Code Examples

```python
def calculate_area(length, width):
    # Preconditions
    assert length > 0 and width > 0, "Length and width must be positive"
    area = length * width
    # Postcondition
    assert area > 0, "Area must be positive"
    return area

print(calculate_area(5, 6))  # OK
print(calculate_area(-5, 6))  # AssertionError
```

When run, the second call raises:

```
AssertionError: Length and width must be positive
```

As explained in tutorial pages. ([TutorialsPoint][12])

### Things to watch/ caveats

* Do **not** rely on asserts for validating user input or expected runtime errors — use proper exceptions for those. Because if Python is run with `-O`, asserts may be skipped and thus your code may continue incorrectly. ([BrowserStack][11])
* If an assertion fails, it will typically stop your program (unless caught). Use only in places where you consider “this condition must always hold if our code is correct”.
* Don’t use asserts for side-effects (because they may be optimized away).
* Overuse of assertions (especially heavy checks) may incur performance overhead in debug mode — be mindful.
* Use clear error messages to help debugging.

### Best Practices

* Use assertions to enforce invariants (e.g., "after this function, this variable must not be None").
* Use them to document assumptions in code paths (makes code more self‐documenting).
* Make sure the error message is descriptive (e.g., `"Expected positive width"`).
* Use them primarily during development and testing; for production checks, use exceptions.
* Keep them simple; don’t do complex logic inside `assert`.

---

## 5. Warnings


Warnings are a mechanism to alert the developer (or user) of a condition in a program that is not immediately fatal but may lead to an issue (deprecated features, best‐practice violations etc). In Python, the `warnings` module provides this facility. ([Python documentation][13])

Important points:

* Warnings are not exceptions (unless configured to be so) — code continues execution after a warning.
* There are built-in warning categories (subclasses of `Warning`), e.g., `DeprecationWarning`, `UserWarning`, `SyntaxWarning`. ([Python documentation][13])
* You can issue a warning via `warnings.warn("message", category=…)`.
* You can filter or control warnings (ignore, always show, convert to error).

### Code Example

```python
import warnings

def old_function(x):
    warnings.warn(
        "old_function() is deprecated; use new_function() instead",
        category=DeprecationWarning,
        stacklevel=2
    )
    return x * 2

print(old_function(5))
```

By default, many `DeprecationWarning`s are ignored unless you run Python with `-Wd` or configure warning filters. You can configure warning behaviour:

```python
import warnings

warnings.simplefilter('always', DeprecationWarning)  # always show
old_function(10)
```

### Things to watch / caveats

* Warnings by default may not be shown (depending on filters and configuration). So if you rely on it, you must set filters appropriately.
* Warning messages should not be used to implement control flow — for fatal errors use exceptions.
* Because code continues execution, warnings might indicate something *wrong* but not handled; being silent about warnings can hide future bugs.
* In production, you may want to log warnings to file or treat them as errors depending on severity.

### Best Practices

* Use warnings when you want to notify about: deprecated behavior, upcoming removal of feature, performance issues, non-fatal misuse.
* Choose an appropriate warning category (e.g., `FutureWarning`, `DeprecationWarning`, `UserWarning`).
* Include helpful message: what’s wrong, what to change, what will happen in future.
* Consider using `stacklevel` parameter so warning reports point to user code location rather than inside your library.
* Consider converting warnings to errors during testing (to avoid missing issues).
* Combine warnings with logging if you need persistent record.

---

## 6. Built-in Exceptions



Python has a large hierarchy of built-in exceptions (classes) which are raised by the interpreter or built-in functions when errors occur. These cover everything from syntax errors, value errors, index errors, file I/O errors, etc. ([Python documentation][4])

Key points:

* All built-in exceptions inherit from `BaseException` (and most user‐level exceptions should inherit from `Exception`). ([Python documentation][4])
* When you write `except SomeException:`, you catch that exception and its subclasses.
* It is good practice to catch specific built-in exceptions rather than broad `except Exception:` or `except:`.
* You can raise any built‐in exception yourself (via `raise ValueError("message")` for example) to signal error conditions. ([Python documentation][4])

### Common Built-In Exceptions (non-exhaustive)

Here are some frequently used ones:

* `ZeroDivisionError` – division / modulo by zero.
* `TypeError` – operation or function applied to object of inappropriate type.
* `ValueError` – built‐in function gets argument of right type but inappropriate value.
* `IndexError` – sequence index out of range.
* `KeyError` – mapping key not found.
* `FileNotFoundError` – file or directory not found.
* `ImportError` / `ModuleNotFoundError` – failed import.
* `OSError` – system‐related errors (e.g., I/O errors).
* `AssertionError` – raised when an `assert` fails.
* `RuntimeError` – generic error when no more specific category fits.
* `StopIteration` – iterator has no more items.

### Code Example

```python
# Example of built-in exceptions
try:
    lst = [1, 2, 3]
    print(lst[5])
except IndexError as e:
    print("Caught IndexError:", e)

try:
    d = {'a': 1}
    print(d['b'])
except KeyError as e:
    print("Caught KeyError:", e)

try:
    x = int("not_a_number")
except ValueError as e:
    print("Caught ValueError:", e)
```

### Best Practices

* Catch the most specific exception you expect. For example, `except IndexError:` rather than `except Exception:` if you know you’re indexing.
* Avoid bare `except:` (which catches `BaseException`, `KeyboardInterrupt`, etc) unless you deliberately want to handle everything (often not advisable).
* Use the exception’s attributes (like `.args`, `.__str__()`) to log or display meaningful information.
* When you catch an exception you cannot handle properly, consider re-raising or converting to another exception or logging it.
* Know the exception hierarchy: catching `Exception` will catch most regular errors but not system‐exit exceptions derived from `BaseException` (e.g., `KeyboardInterrupt`, `SystemExit`) which often should not be suppressed.

---

## Summary Table

| Topic                   | Key Purpose                                | Important Considerations                             |
| ----------------------- | ------------------------------------------ | ---------------------------------------------------- |
| Nested try block        | Handle layered errors at different scopes  | Avoid complexity, maintain readability               |
| User-defined Exceptions | Custom error signalling and handling       | Inherit from `Exception`, meaningful names           |
| Logging                 | Record runtime events, errors, diagnostics | Configure once, use appropriate levels               |
| Assertions              | Internal logic checks during development   | Not for runtime user‐error handling; may be disabled |
| Warnings                | Non-fatal alerts for potential issues      | Code continues; configure filters if needed          |
| Built-in Exceptions     | Standard error types provided by Python    | Catch specific ones, know hierarchy                  |

---


